# Differenced Normalised Burn Ratio for Tumbarumba, NSW <img align="right" src="../DEA reference notebooks/Supplementary_data/dea_logo.jpg">

* **Compatability:** Notebook currently compatible with the `NCI VDI` environment only. Compiled .py script compatible with `NCI Gadi`.
* **Products used:** 
[s2a_ard_granule](https://explorer.sandbox.dea.ga.gov.au/s2a_ard_granule), 
[s2b_ard_granule](https://explorer.sandbox.dea.ga.gov.au/s2b_ard_granule)
* **Prerequisites:** It is recommended that you statistically and visually inspect the available imagery using the `spectral sampling for statistcal analysis` and `Sentinel-2 plotting` notebooks first to get a better idea of the input variables required.

## Background
During the 2020 Black Summer Bushfires, the town of Tumbarumba, NSW was impacted by a fire event that occured on the 05 Jan 2020. By using the analysis ready datasets (Sentinel-2 NBART products) within the DEA Data Cube environments on the NCI, a normalised burn ratio (NBR) for pre and post fire can be calculated in order to create a differnenced normalised burn ratio (dNBR) product. Fire severity and extent can then be inferred from the resulting GeoTIFF that is produced.

## Description
The DEA Data Cube on the NCI is used to access a temporal stack of imagery approximately two months prior and two months after the fire event. It is recommended that you statistically and visually inspect the available imagery using the `spectral sampling for statistcal analysis` and `Sentinel-2 plotting` notebooks first to get a better idea of the input variables required. This notebook is split into the following steps:

1. Import required modules
2. Set required spatiotemporal query variables (location, time)
3. Chunk scene along the x,y dimensions (in-memory control)
4. Calculate dNBR values by iterating spatial chunks
5. Mosaic output rasters into one complete dNBR scene
6. Output the final result to GeoTIFF.

***

## Getting started

### Load packages

In [ ]:
%matplotlib inline

import os
import sys
import datacube
from datacube.helpers import write_geotiff
from datacube.utils import cog
import numpy as np
import pandas as pd
import xarray as xr
import shapely
from shapely.geometry import Point
from matplotlib import pyplot as plt
import glob
import rasterio
from rasterio.merge import merge
from rasterio.plot import show

sys.path.append("/home/554/ab4513/dea-notebooks/Scripts")
import dea_datahandling
from dea_bandindices import calculate_indices
from dea_datahandling import load_ard
from dea_plotting import display_map, rgb

In [ ]:
print(datacube.__version__)

### Connect to the datacube

In [ ]:
dc = datacube.Datacube(app="dNBR")

### Analysis parameters

* `central_lat`: The centroid latitude in decimal degree of the target area of interest (e.g. `-35.783333`).
* `central_lon`: The centroid longitude in decimal degree of the target area of interest (e.g. `148.016667`).
* `crs`: The coordinate reference system code for your I/O tasks (e.g. `EPSG:32755`).
* `prefire_start`: The pre-fire period start date (e.g. `2019-11-01`).
* `prefire_end`: The pre-fire period end date (e.g. `2020-01-06`).
* `postfire_start`: The post-fire period start date (e.g. `2020-01-07`).
* `postfire_end`: The post-fire period start date (e.g. `2020-05-01`).
* `buffer`: The buffer (in arc seconds) around the target coordinates (e.g. `0.2 or 0.4 or 0.6 etc`). Needs to be an even fractional value as per examples.

In [ ]:
# Set the central latitude and longitude
central_lat = -35.783333
central_lon = 148.016667
crs = "EPSG:32755"

# Key Dates
prefire_start = "2019-11-01"
prefire_end = "2020-01-06"
postfire_start = "2020-01-07"
postfire_end = "2020-05-01"

# Set the buffer to load around the central coordinates (even numbers such as 0.2, 1.0, 2.2 etc) in degrees (lat, lon)
buffer = 0.6

# Compute the bounding box for the study area
study_area_lat = (central_lat - buffer, central_lat + buffer)
study_area_lon = (central_lon - buffer, central_lon + buffer)

display_map(x=study_area_lon, y=study_area_lat, margin=-0.2)

### In-Memory I/O Control
Memory allocations on the NCI VDI is 32GB assuming you are the only user on your particular node. As a query to load data is made, data is read into memory. If the time range or spatial extent is too large, a memory allocation erorr will occur. To avoid this, we will create a list of evenly dispersed coordinates and read in smaller spatial chunks at a time. 

> **Note:** The script is designed to chunk the analytical area of interest into 0.2° by 0.2° smaller tiles.

In [ ]:
# Create a range for generating point grid.
magic_number = int(buffer * 10 / 1)
rng = range(int(magic_number / 2))

x_coord = []
y_coord = []

# Calculate x and y for point grid.
for i in rng:
    l = i / 5 + 0.1
    neg_lat = central_lat - l
    neg_lon = central_lon - l
    pos_lat = central_lat + l
    pos_lon = central_lon + l
    x_coord.append(neg_lon)
    x_coord.append(pos_lon)
    y_coord.append(neg_lat)
    y_coord.append(pos_lat)

coords = []

# Create list of shapely points.
for x in x_coord:
    for y in y_coord:
        p = Point(x, y)
        coords.append(p)

### Define and Execute dNBR Calculation
Next we define a function that will interatively query for Sentinel-2 NBART imagery, calculate a dNBR tile and write the output to a local GeoTIFF file store in a subfolder. Once defined, we iterate through all the coordinates to compute all the dNBR tiles.

> **Note:** Using the default notebook parameters on the NCI VDI, this will take approximately 30mins. The NCI VDI allocates users with a 2GB local storage allocation. If you exceed this limit, try reducing your spatial extent or temporal range.

In [ ]:
def dNBR_processing(coordinates):

    # Load all data in baseline period available from s2a/b_ard_granule datasets
    prefire_ard = load_ard(
        dc=dc,
        products=["s2a_ard_granule", "s2b_ard_granule"],
        x=(coordinates.x - 0.1, coordinates.x + 0.1),
        y=(coordinates.y - 0.1, coordinates.y + 0.1),
        time=(prefire_start, prefire_end),
        measurements=["nbart_nir_1", "nbart_swir_3"],
        min_gooddata=0.1,
        output_crs="EPSG:32755",  # UTM Zone 55S
        resolution=(-10, 10),
        group_by="solar_day",
    )

    prefire_ard = calculate_indices(
        prefire_ard, index="NBR", collection="ga_s2_1", drop=False
    )

    # Compute median using all observations in the dataset along the time axis
    prefire_image = prefire_ard.median(dim="time")

    # Delete baseline_combined
    del prefire_ard

    # Select NBR
    prefire_NBR = prefire_image.NBR

    del prefire_image

    # Load all data in post-fire period available from s2a/b_ard_granule datasets
    postfire_ard = load_ard(
        dc=dc,
        products=["s2a_ard_granule", "s2b_ard_granule"],
        x=(coordinates.x - 0.1, coordinates.x + 0.1),
        y=(coordinates.y - 0.1, coordinates.y + 0.1),
        time=(postfire_start, postfire_end),
        measurements=["nbart_nir_1", "nbart_swir_3"],
        min_gooddata=0.1,
        output_crs="EPSG:32755",  # UTM Zone 55S
        resolution=(-10, 10),
        group_by="solar_day",
    )

    # Calculate NBR on all post-fire images
    postfire_ard = calculate_indices(
        postfire_ard, index="NBR", collection="ga_s2_1", drop=False
    )

    # Calculate the median post-fire image
    postfire_image = postfire_ard.median(dim="time")

    del postfire_ard

    # Select NBR
    postfire_NBR = postfire_image.NBR

    del postfire_image

    # Calculate delta
    delta_NBR = prefire_NBR - postfire_NBR

    del prefire_NBR
    del postfire_NBR

    x = np.round_(coordinates.x, decimals=4)
    y = np.round_(coordinates.y, decimals=4)

    # Turn dNBR into a x-array dataset for export to GeoTIFF
    dnbr_dataset = delta_NBR.to_dataset(name="delta_NBR")
    # cog.write_cog(dnbr_dataset, './NBR_geotiffs/{x}_{y}_dNBR.tif')
    write_geotiff(f"./dNBR_geotiffs/{x}_{y}_dNBR.tif", dnbr_dataset)

    del delta_NBR
    del dnbr_dataset

In [ ]:
%%time

# Iterate through all shapely points to generate a dNBR geotiff.
for i in coords:
    dNBR_processing(i)

### Mosaic GeoTIFF tiles into one GeoTIFF dNBR Product
Lastly, we read into all the dNBR tiles that were created and using the Rasterio module, mosaic the tiles into a larger GeoTIFF and save the final product as a GeoTIFF. The dirpath and out_fp locals will need to be changes to your specific files locations. Once complete, you can using SSH to download the final dNBR product.

In [ ]:
dirpath = r"/home/554/ab4513/dea-notebooks/My Notebooks/dNBR_geotiffs"
out_fp = (
    r"/home/554/ab4513/dea-notebooks/My Notebooks/dNBR_geotiffs/Tumbarumba_dNBR.tif"
)

# Make a search criteria to select the DEM files
search_criteria = "1*.tif"
q = os.path.join(dirpath, search_criteria)
print(q)

In [ ]:
dem_fps = glob.glob(q)

In [ ]:
src_files_to_mosaic = []
for fp in dem_fps:
    src = rasterio.open(fp)
    src_files_to_mosaic.append(src)

src_files_to_mosaic

mosaic, out_trans = merge(src_files_to_mosaic)
show(mosaic, cmap="plasma")

In [ ]:
out_meta = src.meta.copy()
out_meta.update(
    {
        "driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_trans,
        "crs": crs,
    }
)

with rasterio.open(out_fp, "w", **out_meta) as dest:
    dest.write(mosaic)

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

Contains modified Copernicus data (2020) processed by Digital Earth Australia.

**Last modified:** 26 Oct 2020.

**Compatible datacube version:**

In [ ]:
print(datacube.__version__)

## Tags